# Zillow Data via Quandl

In [49]:
import os
import sys
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# sys.path.append('C:Users/Rob/repos/labspt15-cityspire-g-ds')
# sys.path

# path = r'C:\Users\Rob\repos\\'
# path == r'V:\\'

In [50]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'

In [51]:
# data = quandl.get_table('MER/F1', compnumber="39102", paginate=True)

In [52]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'
# quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id='99999')

## Data

In [53]:
data = pd.read_csv("V:\ZILLOW_DATA.csv")
data.head()

,indicator_id,region_id,date,value
0,ZATT,3101,1998-01-31,338849.0
1,ZATT,3101,1998-02-28,342993.0
2,ZATT,3101,1998-03-31,346763.0
3,ZATT,3101,1998-04-30,349356.0
4,ZATT,3101,1998-05-31,351981.0


In [54]:
len(data) #125.5 million lines!!

125528409

In [55]:
data['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW',
       'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM', 'CRAW',
       'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW', 'CSSW',
       'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA', 'RSSA'],
      dtype=object)

In [62]:
rental_data = data[data['indicator_id'].isin(['RSSA', 'RSNA'])]

In [63]:
print(len(rental_data))
print(rental_data.shape)
rental_data.head()

535891
(535891, 4)


,indicator_id,region_id,date,value
1586389,RSNA,102001,2020-06-30,1777.0
1586392,RSNA,394913,2020-06-30,2811.0
1586396,RSNA,753899,2020-06-30,2625.0
1586400,RSNA,394463,2020-06-30,1804.0
1586403,RSNA,394514,2020-06-30,1611.0


## Indicators

In [56]:
indicators = pd.read_csv('V:\ZILLOW_INDICATORS.csv')
indicators.head()

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values


In [57]:
indicators['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM',
       'CRAW', 'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW',
       'CSSW', 'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA',
       'RSSA', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW'],
      dtype=object)

In [58]:
indicators

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z1BR,ZHVI 1-Bedroom Time Series ($),Home values
9,LSAW,"Median List Price (Smooth, All Homes, Weekly V...",Inventory and sales


In [65]:
indicators['category'].unique()

array(['Home values', 'Inventory and sales', 'Rentals'], dtype=object)

In [66]:
rentals = indicators[indicators['category'] == 'Rentals']
rentals.head()

,indicator_id,indicator,category
31,RSNA,ZORI (Smoothed): All Homes Plus Multifamily Ti...,Rentals
32,RSSA,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals


In [67]:
for item in rentals['indicator']:
    print(item)

ZORI (Smoothed): All Homes Plus Multifamily Time Series ($)
ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series ($)


## Regions

In [59]:
regions = pd.read_csv('V:\ZILLOW_REGIONS.csv')
regions.head()

,region_id,region_type,region
0,207,county,King County; WA; Seattle-Tacoma-Bellevue
1,791,county,Wayne County; MI; Detroit-Warren-Dearborn
2,3136,county,Santa Clara County; CA; San Jose-Sunnyvale-San...
3,3175,county,Philadelphia County; PA; Philadelphia-Camden-W...
4,92515,zip,78572; TX; McAllen-Edinburg-Mission; Hidalgo C...


In [60]:
len(regions)

78730

In [61]:
regions['region_type'].unique()

array(['county', 'zip', 'metro', 'state', 'neigh', 'city'], dtype=object)

In [68]:
len(regions['region_type'] == 'metro')

78730

In [69]:
metro = regions[regions['region_type'] == 'metro']
metro[:50]

,region_id,region_type,region
7,394913,metro,"New York, NY"
8,753899,metro,"Los Angeles-Long Beach-Anaheim, CA"
9,394463,metro,"Chicago, IL"
12,394514,metro,"Dallas-Fort Worth, TX"
40,394692,metro,"Houston, TX"
41,394856,metro,"Miami-Fort Lauderdale, FL"
42,395057,metro,"San Francisco, CA"
43,394532,metro,"Detroit, MI"
44,395121,metro,"St. Louis, MO"
45,395148,metro,"Tampa, FL"


# Data.World Population Data Set

In [70]:
population = pd.read_excel('https://query.data.world/s/gewikohoj4s2gqv4bowlzpdxeu4jzl', usecols=['State', 'City', 'Population'])
population.head()

,State,City,Population
0,Alabama,Abernant,7599
1,Alabama,Alabaster,31545
2,Alabama,Albertville,21458
3,Alabama,Alexander City,14849
4,Alabama,Andalusia,9081


In [73]:
len(population)

6889

In [74]:
population['City']

0             Abernant
1            Alabaster
2          Albertville
3       Alexander City
4            Andalusia
             ...      
6884            Lander
6885        Torrington
6886           Douglas
6887            Powell
6888    Bridger Valley
Name: City, Length: 6889, dtype: object

In [83]:
cities = population[population['Population'] > 80000]

In [84]:
len(cities['State'].unique())

47

In [85]:
cities['State'].unique()

array(['Alabama', 'Alaska', 'Arkansas', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'DC', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Virginia', 'Washington',
       'Wisconsin'], dtype=object)

In [86]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire','New Jersey', 
'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
'West Virginia', 'Wisconsin', 'Wyoming']

In [87]:
missing = []

for state in states:
    if state not in list(cities['State'].unique()):
        missing.append(state)

print(missing)

['Maine', 'Vermont', 'West Virginia', 'Wyoming']


# Aggregating Data

In [ ]:
# Checking if data has all the same sections
sorted(data['indicator_id'].unique()) == sorted(indicators['indicator_id'].unique())